> #  __Neighborhood Evaluator__  
      In this project, we will be using Foursquare's data location set to help locate a suitable neighborhood to move to that is:  
          1. closer to a newly accepted job  
          2. is very similar to our current neighborhood  
          3. determine how popular it is  

In [1]:
# Notebook is used for the IBM Data Science capstone project

import pandas as pd
import numpy as np
print('Imported')

Imported


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
# Cells 25-26 are to answer the question: "Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe."

df = pd.read_csv("Toronto.csv")
print(df.head())

  Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae


In [4]:
# All lines with the Borough and/or Neighborhood listed as "Not Assigned" were not included in the data set.

df.shape

(103, 3)

In [5]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [6]:
import geocoder # import geocoder
print('Imported')

Imported


In [7]:
# imports Toronto data with coordinates assigned to answer "Use the Geocoder package or the csv file to create the following dataframe:"

TorontoCoordinates = pd.read_csv("TorontoCoordinates.csv")
print(TorontoCoordinates.head())

  Postal Code      Borough                            Neighborhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


In [8]:
print('The dataframe has {} boroughs.'.format(
        len(TorontoCoordinates['Borough'].unique()))
     )

The dataframe has 10 boroughs.


In [9]:
# Create new table with only the Boroughs witin Toronto only

TorontoBoroughs = TorontoCoordinates[TorontoCoordinates['Borough'].str.contains('Toronto',regex=False)]
print(TorontoBoroughs.head())

   Postal Code          Borough                    Neighborhood   Latitude  \
37         M4E     East Toronto                     The Beaches  43.676357   
41         M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42         M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
43         M4M     East Toronto                 Studio District  43.659526   
44         M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  
37 -79.293031  
41 -79.352188  
42 -79.315572  
43 -79.340923  
44 -79.388790  


In [10]:
# import plotting library

import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')

Folium installed


In [11]:
# Create a map of Toronto using the Boroughs within Toronto, excluding those outside of Toronto

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,Borough,Neighborhood in zip(TorontoBoroughs['Latitude'],TorontoBoroughs['Longitude'],TorontoBoroughs['Borough'],TorontoBoroughs['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [12]:
k=5
toronto_kdf = TorontoBoroughs.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_kdf)
kmeans.labels_
TorontoBoroughs.insert(0, 'Cluster Labels', kmeans.labels_)

In [13]:
print(TorontoBoroughs.head())

    Cluster Labels Postal Code          Borough  \
37               0         M4E     East Toronto   
41               0         M4K     East Toronto   
42               0         M4L     East Toronto   
43               0         M4M     East Toronto   
44               1         M4N  Central Toronto   

                      Neighborhood   Latitude  Longitude  
37                     The Beaches  43.676357 -79.293031  
41    The Danforth West, Riverdale  43.679557 -79.352188  
42  India Bazaar, The Beaches West  43.668999 -79.315572  
43                 Studio District  43.659526 -79.340923  
44                   Lawrence Park  43.728020 -79.388790  


In [14]:
# Create the cluster map

toronto_cluster_map = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(TorontoBoroughs['Latitude'], TorontoBoroughs['Longitude'], TorontoBoroughs['Neighborhood'], TorontoBoroughs['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_cluster_map)
toronto_cluster_map